In [1]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

[] Her otobüsün tek bir defa çalışma kısıtı eklenmeli
[]

In [2]:
# Parametreler
sofor_i = 3
otobus_j = 1
hat_k = 2
sefer_l = 2
gun_y = 1

# T_k: k. hatta ihtiyaç duyulan şoför sayısı
# W_kl: k. hattın l. vardiyasındaki sefer sayısı
T_k = [1, 1, 1]

In [3]:
# i: şoför 1,…,n
# j: gün 1, …, m
# k: hat 1, …, p
# l: vardiya 1, …, t

sofor_sayisi_i = range(1, sofor_i + 1)
otobus_sayisi_j = range(1, otobus_j + 1)
hat_sayisi_k = range(1, hat_k + 1)
sefer_sayisi_l = range(1, sefer_l + 1)
gun_sayisi_y = range(1, gun_y + 1)

In [4]:
X = {}

for i in sofor_sayisi_i:
    for j in otobus_sayisi_j:
        for y in gun_sayisi_y:
            for k in hat_sayisi_k:
                for l in sefer_sayisi_l:
                    X[(i, j, y, k, l)] = model.NewBoolVar('X_i%i-j%i-k%i-l%i-y%i' % (i, j, y, k, l))

X

{(1, 1, 1, 1, 1): X_i1-j1-k1-l1-y1(0..1),
 (1, 1, 1, 1, 2): X_i1-j1-k1-l1-y2(0..1),
 (1, 1, 1, 2, 1): X_i1-j1-k1-l2-y1(0..1),
 (1, 1, 1, 2, 2): X_i1-j1-k1-l2-y2(0..1),
 (2, 1, 1, 1, 1): X_i2-j1-k1-l1-y1(0..1),
 (2, 1, 1, 1, 2): X_i2-j1-k1-l1-y2(0..1),
 (2, 1, 1, 2, 1): X_i2-j1-k1-l2-y1(0..1),
 (2, 1, 1, 2, 2): X_i2-j1-k1-l2-y2(0..1),
 (3, 1, 1, 1, 1): X_i3-j1-k1-l1-y1(0..1),
 (3, 1, 1, 1, 2): X_i3-j1-k1-l1-y2(0..1),
 (3, 1, 1, 2, 1): X_i3-j1-k1-l2-y1(0..1),
 (3, 1, 1, 2, 2): X_i3-j1-k1-l2-y2(0..1)}

In [5]:
# Kısıtlar

In [6]:
for y in gun_sayisi_y:
    for k in hat_sayisi_k:
        for l in sefer_sayisi_l:
            model.Add(sum(X[(i, j, y, k, l)] for i in sofor_sayisi_i for j in otobus_sayisi_j) == 1)

In [7]:


min_sefer_sofor = (otobus_j * gun_y * hat_k * sefer_l) // sofor_i

if (otobus_j * gun_y * hat_k * sefer_l) % sofor_i == 0:
    max_sefer_sofor = min_sefer_sofor
else:
    max_sefer_sofor = min_sefer_sofor + 1

for i in sofor_sayisi_i:
    shifts_worked = []
    for j in otobus_sayisi_j:
        for y in gun_sayisi_y:
            for k in hat_sayisi_k:
                for l in sefer_sayisi_l:
                    shifts_worked.append(X[(i, j, y, k, l)])
    model.Add(min_sefer_sofor <= sum(shifts_worked))
    model.Add(sum(shifts_worked) <= max_sefer_sofor)

In [8]:
class DriversSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, X, sofor_i, otobus_j, hat_k, sefer_l, gun_y, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._X = X
        self._sofor_i = sofor_i + 1
        self._otobus_j = otobus_j + 1
        self._hat_k = hat_k + 1
        self._sefer_l = sefer_l + 1
        self._gun_y = gun_y + 1
        self._solution_count = 0
        self._solution_limit = limit
        self._solution_json = {}





    def on_solution_callback(self):
        self._solution_count += 1
        solution_list = []
        print('Çözüm %i' % self._solution_count)
        for y in range(1, self._gun_y):
            print('  Gün %i' % y)

            for i in range(1, self._sofor_i):
                is_working = False
                for j in range(1, self._otobus_j):
                    for k in range(1, self._hat_k):
                        for l in range(1, self._sefer_l):
                            if self.Value(self._X[(i, j, y, k, l)]):
                                is_working = True
                                print('    Şoför %i -> %i. otobüs, %i. hat, %i. sefer' % (i, j, k, l))
                if not is_working:
                    print('    Şoför {} çalışmıyor.'.format(i))


        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()


    def solution_count(self):
        return self._solution_count


solution_limit = 1
solution_printer = DriversSolutionPrinter(X, sofor_i, otobus_j, hat_k, sefer_l, gun_y, solution_limit)

In [9]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

In [10]:
f = solver.Solve(model, solution_printer)

Çözüm 1
  Gün 1
    Şoför 1 -> 1. otobüs, 2. hat, 1. sefer
    Şoför 1 -> 1. otobüs, 2. hat, 2. sefer
    Şoför 2 -> 1. otobüs, 1. hat, 2. sefer
    Şoför 3 -> 1. otobüs, 1. hat, 1. sefer
Stop search after 1 solutions


In [14]:
f

2